In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
EPOCHS = 15
#YEAR = '2000'
YEAR = '2010'

df = pd.read_csv('./data/census/census_%s.csv' % YEAR)
df.dropna(subset=['name'], inplace=True)
df.replace('(S)', 0, inplace=True)
df

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4
1,JOHNSON,2,1932812,655.24,1483.42,58.97,34.63,0.54,0.94,2.56,2.36
2,WILLIAMS,3,1625252,550.97,2034.39,45.75,47.68,0.46,0.82,2.81,2.49
3,BROWN,4,1437026,487.16,2521.56,57.95,35.6,0.51,0.87,2.55,2.52
4,JONES,5,1425470,483.24,3004.80,55.19,38.48,0.44,1,2.61,2.29
5,GARCIA,6,1166120,395.32,3400.12,5.38,0.45,1.41,0.47,0.26,92.03
6,MILLER,7,1161437,393.74,3793.86,84.11,10.76,0.54,0.66,1.77,2.17
7,DAVIS,8,1116357,378.45,4172.31,62.2,31.6,0.49,0.82,2.45,2.44
8,RODRIGUEZ,9,1094924,371.19,4543.50,4.75,0.54,0.57,0.18,0.18,93.77
9,MARTINEZ,10,1060159,359.40,4902.90,5.28,0.49,0.6,0.51,0.22,92.91


## Resampling with weight

In [2]:
sdf = df.sample(1000000, weights=df['count'], replace=True)

## Assign race by pertcentage

In [3]:
from numpy.random import choice

races = ['white', 'black', 'api', 'hispanic']

def to_race(c):
    w = np.array(c).astype(float)
    probs = w/w.sum()
    return choice(races, p=probs)

sdf['race'] = sdf[['pctwhite', 'pctblack', 'pctapi', 'pcthispanic']].apply(lambda c: to_race(c), axis=1)
sdf

/opt/venv/lib/python2.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in divide
  import sys
/opt/venv/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  


,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic,race
77312,SHULKIN,77264,248,0.08,85625.76,92.74,0,0,0,2.42,3.23,white
351,MCDANIEL,352,88728,30.08,29075.42,76.18,18.35,0.43,0.64,2.05,2.35,white
162253,ALL OTHER NAMES,0,29312001,9936.97,9936.97,66.65,8.53,7.97,0.86,2.32,13.67,white
49,ROBERTS,50,376774,127.73,12608.39,77.27,16.72,0.61,0.85,2.15,2.4,white
162253,ALL OTHER NAMES,0,29312001,9936.97,9936.97,66.65,8.53,7.97,0.86,2.32,13.67,hispanic
102802,HALMA,102688,175,0.06,87420.41,95.43,0,0,0,0,0,white
14525,CAPASSO,14523,2053,0.70,72301.13,93.67,0.39,0,0,1.07,4.68,white
142,MCDONALD,143,180497,61.19,20250.34,76.65,17.28,0.55,0.78,2.01,2.73,white
668,GLENN,669,51043,17.30,36251.61,61.41,32.78,0.48,0.63,2.39,2.32,black
120246,BLONDO,120187,144,0.05,88359.83,91.67,0,0,0,0,7.64,white


## Check the correctness of race assignment

In [4]:
df[df.name == 'SMITH']

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2442977,828.19,828.19,70.9,23.11,0.5,0.89,2.19,2.4


In [5]:
xdf = sdf[sdf.name=='SMITH'].groupby(['race']).agg({'name': 'count'})
xdf * 100 / xdf.sum()

,name
race,
api,0.585866
black,23.739778
hispanic,2.367875
white,73.306481


In [6]:
# Additional features
sdf['name_last'] = sdf.name.str.title()
sdf.groupby('race').agg({'name_last': 'count'})

,name_last
race,
api,50354
black,124086
hispanic,164911
white,660649


In [7]:
len(sdf)

1000000

## Preprocessing the input data

In [8]:
# only last name in Census data
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
#vect = CountVectorizer(analyzer='char', ngram_range=(2, 2), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_
len(vocab)

981

In [9]:
import operator
sorted_vocab = sorted(vocab.items(), key=operator.itemgetter(1))
cols = list(map(operator.itemgetter(0), sorted_vocab))

In [10]:
count_df = pd.DataFrame(a.todense(), columns=cols)
count_df

,N,O,Aa,Ab,Ac,Ad,Ae,Af,Ag,Ah,...,zp,zq,zr,zs,zt,zu,zv,zw,zy,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
count_df.sum().sort_values(ascending=False).describe()

count       981.000000
mean       6297.886850
std       18172.689125
min           3.000000
25%          74.000000
50%         832.000000
75%        4959.000000
max      253968.000000
dtype: float64

In [12]:
pd.set_option('display.max_rows', 20)
count_df.sum().sort_values(ascending=False)

er    253968
ll    174973
es    144539
am    132284
he    124455
th    119284
me    115286
on    109332
Al    108394
Na    102706
       ...  
Ij         3
qa         3
Yd         3
Gd         3
fq         3
Tv         3
Uf         3
jb         3
Yn         3
Fn         3
Length: 981, dtype: int64

In [13]:
# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)

num_words = 981


In [14]:
def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

Max feature len = 14, Avg. feature len = 6


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [15]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

/opt/venv/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


In [16]:
print('Build model...')

model = Sequential()
model.add(Embedding(num_words, 32, input_length=feature_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 32)            31392     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 114,340
Trainable params: 114,340
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=2)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 720000 samples, validate on 80000 samples
Epoch 1/15
 - 1703s - loss: 0.6550 - acc: 0.7760 - val_loss: 0.6267 - val_acc: 0.7851
Epoch 2/15
 - 1699s - loss: 0.6175 - acc: 0.7888 - val_loss: 0.6067 - val_acc: 0.7916
Epoch 3/15
 - 1703s - loss: 0.6033 - acc: 0.7931 - val_loss: 0.5974 - val_acc: 0.7946
Epoch 4/15
 - 1701s - loss: 0.5955 - acc: 0.7954 - val_loss: 0.5912 - val_acc: 0.7959
Epoch 5/15
 - 1655s - loss: 0.5908 - acc: 0.7964 - val_loss: 0.5851 - val_acc: 0.7971
Epoch 6/15
 - 1641s - loss: 0.5874 - acc: 0.7974 - val_loss: 0.5834 - val_acc: 0.7991
Epoch 7/15
 - 1489s - loss: 0.5854 - acc: 0.7980 - val_loss: 0.5819 - val_acc: 0.7997
Epoch 8/15
 - 1382s - loss: 0.5837 - acc: 0.7985 - val_loss: 0.5794 - val_acc: 0.7997
Epoch 9/15
 - 1370s - loss: 0.5821 - acc: 0.7989 - val_loss: 0.5790 - val_acc: 0.7998
Epoch 10/15
 - 1039s - loss: 0.5813 - acc: 0.7990 - val_loss: 0.5776 - val_acc: 0.8000
Epoch 11/15
 - 601s - loss: 0.5809 - acc: 0.7993 - val_loss: 0.5760 - val_acc: 

## Confusion Matrix

In [18]:
y_pred = model.predict_classes(X_test, verbose=2)
p = model.predict_proba(X_test, verbose=2) # to predict probability
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

        api       0.84      0.57      0.68     10071
      black       0.57      0.07      0.12     24817
   hispanic       0.88      0.76      0.82     32982
      white       0.79      0.97      0.87    132130

avg / total       0.78      0.80      0.76    200000

[[  5743     42    796   3490]
 [   257   1693    218  22649]
 [   173     82  25118   7609]
 [   694   1157   2442 127837]]


## Save model

In [19]:
model.save('./models/census/lstm/census%s_ln_lstm.h5' % YEAR)

In [20]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./models/census/lstm/census%s_ln_vocab.csv' % YEAR, index=False, encoding='utf-8')